In [ ]:
BiocManager::install('glmGamPoi')
remotes::install_github("jinworks/CellChat@b02aaaf")

In [ ]:
suppressPackageStartupMessages({
    library(Seurat)
    library(future)
    library(dplyr)
    library(ComplexHeatmap)
    library(RColorBrewer)
    library(circlize)
    library(ggplot2)
    library(colorspace)
    library(ggplot2)
    library(dplyr)
    library(Matrix)
    library(CellChat)
    library(RColorBrewer)
    library(zellkonverter)
})

In [ ]:
sce = readH5AD("tmp/cellchat_preparation_human_signature.h5")
total_seurat = as.Seurat(sce, counts = "X", data = NULL)

In [ ]:
options(future.globals.maxSize= 10091289600)
Idents(total_seurat) <- "Immunocentric_Type_With_P14"
total_seurat <- subset(total_seurat, idents = c('B-Cell', 'CD4', 'CD8 GZMK+ Effector', 'CD8 T-Cell',
       'CD8AB+ Effector Like', 'CD8AB+ Stem Like', 'Conventional_DC',
       'Effector T-Cell', 'Enterocyte', 'Enteroendocrine', 'Fibroblast',
       'Goblet', 'Lymphatic', 'Macrophage', 'Mast', 'Monocyte', 'Neuron',
       'Other_DC', 'Other_Myeloid', 'PP_CD4_TCell', 'PP_CD8AB_TCell',
       'PP_CD8AB_Tcell_GZMK+', 'PP_CD8AB_Tcell_Proliferating', 'PP_TCell',
       'Paneth', 'PlasmaCell', 'Proliferating_Myeloid',
       'Proliferating_Tcell', 'T-Cell_GZMK+_ITGAE+', 'TCell_other_11',
       'TCell_other_13', 'Transit_Amplifying', 'Treg', 'Tuft',
       'Vascular_Endothelia', 'Vascular_Endothelia_AQP1+',
       'Vascular_Endothelia_CPE+', 'cDC1', 'gd_TCell'))
total_seurat <- SCTransform(total_seurat, assay = "originalexp", clip.range = c(-10, 10))

# Create and run cellchat

In [ ]:
options(stringsAsFactors = FALSE)

#Prepare input data for CellChat analysis

data.input = GetAssayData(total_seurat, slot = "data", assay = "SCT") # normalized data matrix


meta = data.frame(labels = Idents(total_seurat), row.names = names(Idents(total_seurat))) # manually create a dataframe consisting of the cell labels

### load spatial imaging information
# Spatial locations of spots from full (NOT high/low) resolution images are required
#spatial.locs = GetTissueCoordinates(duo_d7, scale = NULL, cols = c("imagerow", "imagecol"))
#alternative from Merscope Seurat object
spatial.locs <- total_seurat@meta.data %>%
  dplyr::select(x, y) %>%
  dplyr::rename(
    imagerow = x,
    imagecol = y
  )

In [ ]:
### Scale factors and spot diameters of the full resolution images 
#This is different than the tutorial - see https://github.com/sqjin/CellChat/issues/514

avg.diameter <- 5

#scale.factors
scale.factors <- list(spot.diameter = avg.diameter, spot = 5)

#####Create CellChat Object
total_seurat_cellchat <- createCellChat(object = data.input, meta = meta, group.by = "labels",
                           datatype = "spatial", coordinates = spatial.locs, scale.factors = scale.factors)

# Set the ligand-receptor interaction database

CellChatDB <- CellChatDB.human # use CellChatDB.human if running on human data

# use a subset of CellChatDB for cell-cell communication analysis
#CellChatDB.use <- subsetDB(CellChatDB, search = "Secreted Signaling") # use Secreted Signaling

# use all CellChatDB for cell-cell communication analysis
CellChatDB.use <- CellChatDB # simply use the default CellChatDB

# set the used database in the object
total_seurat_cellchat@DB <- CellChatDB.use

In [ ]:


### Preprocessing the expression data for cell-cell communication analysis

# subset the expression data of signaling genes for saving computation cost
total_seurat_cellchat <- subsetData(total_seurat_cellchat) # This step is necessary even if using the whole database
total_seurat_cellchat <- identifyOverExpressedGenes(total_seurat_cellchat)
total_seurat_cellchat <- identifyOverExpressedInteractions(total_seurat_cellchat)




In [ ]:
total_seurat_cellchat <- projectData(total_seurat_cellchat, PPI.human)

In [ ]:
# project gene expression data onto PPI (Optional: when running it, USER should set `raw.use = FALSE` in the function `computeCommunProb()` in order to use the projected data)
total_seurat_cellchat <- computeCommunProb(total_seurat_cellchat, type = "truncatedMean", trim = 0.1, distance.use = TRUE, interaction.range = 50, scale.distance = 1)


In [ ]:
# Filter out the cell-cell communication if there are only few number of cells in certain cell groups
total_seurat_cellchat <- filterCommunication(total_seurat_cellchat, min.cells = 10)

#Infer the cell-cell communication at a signaling pathway level
total_seurat_cellchat <- computeCommunProbPathway(total_seurat_cellchat)

#Calculate the aggregated cell-cell communication network
total_seurat_cellchat <- aggregateNet(total_seurat_cellchat)

In [ ]:
#Here we take input of one signaling pathway as an example. All the signaling pathways showing significant communications can be accessed by:
signif.pathways <- total_seurat_cellchat@netP$pathways

# Compute the network centrality scores
total_seurat_cellchat <- netAnalysis_computeCentrality(total_seurat_cellchat, slot.name = "netP") # the slot 'netP' means the inferred intercellular communication network of signaling pathways
# Visualize the computed centrality scores using heatmap, allowing ready identification of major signaling roles of cell groups

# save Object
saveRDS(total_seurat_cellchat, file = "tmp/total_seurat_human_cellchat_signature.rds")

# Session Info
sessionInfo()